# Install Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import uuid
import json
import os
import re
import sys
from tqdm import tqdm
from tika import parser
import markdown
from pathlib import Path
from html.parser import HTMLParser

# Identifying the target pages

In [2]:
worker_pages = ['הודעה_מוקדמת_לפיטורים','מנוחה_שבועית','שוויון_בעבודה','הפסקות_בעבודה',
                'יום_עבודה_ושבוע_עבודה','זמן_תפילה_בעבודה',
                'ערעור_על_פיטורים','העסקת_נשים_במשמרת_לילה','מענק_שחרור_לחיילים_משוחררים_ומסיימי_שירות_לאומי-אזרחי',
                'דמי_אבטלה_לחיילים_משוחררים_ומסיימי_שירות_לאומי-אזרחי','גמול_עבור_שעות_נוספות',
                'שכר_שווה','תשלום_דמי_חופשה','איסור_העסקת_נוער_בעבודת_לילה','איסור_התעמרות_בעבודה','מניעת_עישון_במקום_העבודה','חופשה_שנתית',
                'פיצויי_פיטורים_לעובד_שמעסיקו_נפטר',]

pension_pages = ['גיל_פרישה_מעבודה','קרן_פנסיה','איסור_פיטורים_בגיל_פרישה_של_אלמנות_צה"ל_והורים_שכולים_שילדם_נפטר_עקב_שירות_צבאי',
                 'ביטוח_מנהלים','קופת_גמל_להשקעה','חובת_ביטוח_פנסיוני_לעובדים','קרן_השתלמות',
                 'ביטוח_פנסיוני_לעובד_עצמאי','זכותון_קצבת_זיקנה','השכר_הממוצע','כיסוי_ביטוחי_במסגרת_הביטוח_הפנסיוני']

housing_pages = ['משכנתא_בנקאית_(מסחרית)','ביטוח_משכנתא','קבלת_משכנתא_לזכאים','מגבלות_על_לקיחת_משכנתא','ביטוח_משכנתא','קבלת_משכנתא_לזכאים',
                 'פירעון_מוקדם_של_מלוא_סכום_המשכנתא','מיחזור_משכנתא',
                 'דירה_בהנחה_(מחיר_למשתכן)','דמי_מפתח','חוזה_שכירות_למגורים','הגבלת_סכום_הערובה_שמותר_לדרוש_משוכר_דירה',
                 'ביטול_חד-צדדי_של_חוזה_שכירות_שלא_עקב_הפרתו','תשלומי_ועד_בית','תשלומים_שאסור_להטיל_על_שוכר_דירה','תשלומים_שמותר_להטיל_על_שוכר_דירה',
                 'קבלת_סיוע_בשכר_דירה','חישוב_מס_רכישה','דירת_מגורים_מזכה','חישוב_מס_שבח','שומת_מס_שבח','תשלום_מס_שבח','היטל_השבחה']

education_pages = ['מעון_יום_שיקומי','סיוע_להורים_לומדים_במימון_מעונות_יום_ומשפחתונים','מעונות_יום_ומשפחתונים_לפעוטות_בסיכון',
                  'הזכות_לחינוך','זכות_התלמיד_לכבוד_ולפרטיות','קבלת_מידע_מבית_ספר_ממלכתי','איסור_פגיעה_בתלמיד_בשל_אי_תשלום_כספים_על-ידי_הוריו',
                   'איסור_אפליה_בחינוך_מטעמים_של_מוצא_עדתי','הענשת_תלמיד_בגין_הופעה_בלתי_הולמת_או_ללא_תלבושת_אחידה_בבית_ספר',
                   'הענשת_תלמיד_בגין_שימוש_בטלפון,_טאבלט_או_מצלמה_בזמן_שיעור_בבית_הספר','פסילת_בחינה_של_תלמיד_בבית_הספר_בגין_אי_שמירה_על_טוהר_הבחינות',
                   'הענשת_תלמיד_בית_ספר_יסודי_בגין_עישון_בבית_הספר','הענשת_תלמיד_בית_ספר_על-יסודי_בגין_עישון_בבית_הספר',
                   'הענשת_תלמיד_בגין_הבאה_או_שתייה_של_אלכוהול_בבית_הספר','דרכי_ענישה_אסורות_בבית_הספר',
                   'מניעת_כניסה_של_תלמיד_לשיעור','הרחקת_תלמיד_מבית_הספר_(השעיה)','העברת_תלמיד_לכיתה_מקבילה_באופן_קבוע',
                   'הטלת_עונש_כספי_בשל_פגיעה_ברכוש_בית_הספר','בדיקות_שיגרה_לתלמידי_בית_ספר','חיסונים_לתלמידי_בית_ספר','מתן_טיפול_תרופתי_במוסד_החינוך',
                   'ארוחה_יומית_לתלמיד','איכות_המזון_בצהרונים','הזכות_להיבחן_בבחינת_הבגרות','שלילת_הזכאות_להיבחן_בבחינת_הבגרות',
                   'ערעור_על_שלילת_הזכאות_להיבחן_בבחינת_הבגרות','שיפור_ציוני_בגרות_ומועד_בגרות_מיוחד_בתוכנית_סמסטר_קיץ',
                   'תשלומי_חובה_במערכת_החינוך']

# this pages will be for the testing part
validation_pages = ['הקפאת_ביציות','דמי_מחלה','ימי_מחלה',
                    'חופשת_לידה_(תקופת_לידה_והורות)','סחיטה_באיומים_בעלת_אופי_מיני','מימון_עלויות_פינוי_באמבולנס',
                    'קצבת_שאירים','הפקדת_זרע','קצבת_נכות_כללית']


testing_pages = ['מדריך_להתמודדות_עם_הטרדה_מינית_במסגרת_העבודה','איסור_פיטורי_עובדת_בהיריון','אימוץ_ילד_בישראל'
                 ,'חופשת_לידה_להורה_מאמץ_(חופשת_אימוץ)','קבלת_תרומת_זרע'
                 ,'קבלת_תרומת_ביצית','קצבת_לידה','החזר_הוצאות_נסיעה']



In [3]:
# create template dataframes
train_context_df=pd.DataFrame(columns=["title","paragraphs"])
valid_context_df=pd.DataFrame(columns=["title","paragraphs"])
test_context_df=pd.DataFrame(columns=["title","paragraphs"])

In [4]:
# extract paragraphs from each page where the paragraph text size is bigger than 500 and has a point meaning consistence of sentences
for title in worker_pages+pension_pages+housing_pages+education_pages:
    paragraphs=[]
    url=f"https://www.kolzchut.org.il/he/{title}"
    page=requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    for tag in soup.find_all('ul'):
        if(len(tag.text)>500 and '.' in tag.text):
            paragraphs.append(tag.text)
    train_context_df.loc[len(train_context_df.index)] = [title, paragraphs]
    print(train_context_df.shape)

(1, 2)
(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)
(8, 2)
(9, 2)
(10, 2)
(11, 2)
(12, 2)
(13, 2)
(14, 2)
(15, 2)
(16, 2)
(17, 2)
(18, 2)
(19, 2)
(20, 2)
(21, 2)
(22, 2)
(23, 2)
(24, 2)
(25, 2)
(26, 2)
(27, 2)
(28, 2)
(29, 2)
(30, 2)
(31, 2)
(32, 2)
(33, 2)
(34, 2)
(35, 2)
(36, 2)
(37, 2)
(38, 2)
(39, 2)
(40, 2)
(41, 2)
(42, 2)
(43, 2)
(44, 2)
(45, 2)
(46, 2)
(47, 2)
(48, 2)
(49, 2)
(50, 2)
(51, 2)
(52, 2)
(53, 2)
(54, 2)
(55, 2)
(56, 2)
(57, 2)
(58, 2)
(59, 2)
(60, 2)
(61, 2)
(62, 2)
(63, 2)
(64, 2)
(65, 2)
(66, 2)
(67, 2)
(68, 2)
(69, 2)
(70, 2)
(71, 2)
(72, 2)
(73, 2)
(74, 2)
(75, 2)
(76, 2)
(77, 2)
(78, 2)
(79, 2)
(80, 2)
(81, 2)


In [5]:
for title in validation_pages:
    paragraphs=[]
    url=f"https://www.kolzchut.org.il/he/{title}"
    page=requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    for tag in soup.find_all('ul'):
        if(len(tag.text)>500 and '.' in tag.text):
            paragraphs.append(tag.text)
    valid_context_df.loc[len(valid_context_df.index)] = [title, paragraphs]
    print(valid_context_df.shape)

(1, 2)
(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)
(8, 2)
(9, 2)


In [6]:
for title in testing_pages:
    paragraphs=[]
    url=f"https://www.kolzchut.org.il/he/{title}"
    page=requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    for tag in soup.find_all('ul'):
        if(len(tag.text)>500 and '.' in tag.text):
            paragraphs.append(tag.text)
    test_context_df.loc[len(test_context_df.index)] = [title, paragraphs]
    print(test_context_df.shape)

(1, 2)
(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)
(8, 2)


In [7]:
def df2squadFormat(df, output_dir=None, filename=None):
    """
    Convert pandas DataFrame to SQuAD format for annotation process.
    
    Args:
        df (DataFrame): Input DataFrame containing 'title' and 'paragraphs' columns.
        output_dir (str): Output directory to save the JSON file.
        filename (str): Filename for the output JSON file.
    
    Returns:
        dict: JSON data in the SQuAD format.
    """
    json_data = {}
    json_data["data"] = []

    for idx, row in tqdm(df.iterrows()):
        temp = {"title": row["title"], "paragraphs": []}
        for paragraph in row["paragraphs"]:
            temp["paragraphs"].append({"context": paragraph, "qas": []})
        json_data["data"].append(temp)

    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        with open(os.path.join(output_dir, "{}.json".format(filename)), "w") as outfile:
            json.dump(json_data, outfile)

    return json_data

In [8]:
json_train_data = df2squadFormat(df=train_context_df, output_dir='.', filename='rights-train-before-annotation')
json_valid_data = df2squadFormat(df=valid_context_df, output_dir='.', filename='rights-valid-before-annotation')
json_test_data = df2squadFormat(df=test_context_df, output_dir='.', filename='rights-test-before-annotation')

81it [00:00, 16102.88it/s]
9it [00:00, 4007.72it/s]
8it [00:00, 8021.62it/s]


In [9]:
json_train_data

{'data': [{'title': 'הודעה_מוקדמת_לפיטורים',
   'paragraphs': [{'context': 'יחסי העובד-מעסיק בין הצדדים מסתיימים עם תום תקופת ההודעה המוקדמת (כלומר, בסיום יחסי העבודה בפועל).\nבמהלך התקופה שמרגע מתן הודעת הפיטורים ועד סיום העבודה, המעסיק חייב להעניק לעובד את כל תנאי העבודה שהיה זכאי להם לפני כן.\nהעובד מחויב במהלך אותה תקופה להעמיד את עצמו לרשות העבודה ולהמשיך לעבוד באותו היקף ובאותה רצינות, כפי שנדרשו ממנו לפני כן.\nהחובה לתת לעובד הודעה מוקדמת  חלה גם בעת פיטורים עקב חילופי מעסיקים. חובה זו חלה  גם אם העובד ממשיך לעבוד באותו מקום עבודה אצל המעסיק החדש. לקריאת פסק דין בסוגיה זו ראו בקישור הזה.',
     'qas': []},
    {'context': 'על פי סעיף 5 לחוק חופשה שנתית, ימי הודעה מוקדמת לפיטורים לא יבואו במניין ימי החופשה אלא אם עלו על 14 ימים. כלומר, 14 ימים מתוך תקופת ההודעה המוקדמת לא יחפפו את תקופת החופשה ויש להבטיח כי לפחות 14 ימים מתוך תקופת ההודעה המוקדמת לא ינוצלו לחופשה.\nעל פי  פסיקת בית הדין הארצי לעבודה, כאשר המעסיק מוותר  על עבודתו של העובד בתקופת ההודעה המוקדמת, אסור לו לחפוף את תק

In [10]:
json_valid_data

{'data': [{'title': 'הקפאת_ביציות',
   'paragraphs': [{'context': 'סיבות רפואיות שהוכרו כמצדיקות הקפאת ביציות:\nבמהלך טיפולי פריון:\nבזוגות אצלם יש לגבר הפרעה קשה באיכות ו/או ספירת הזרע;\nנשים עם תגובה ירודה - יצירת מאגר ביציות;\nחוסר יכולת לתת זרע ביום השאיבה;\nאנדומטריוזיס קשה;\nמורסה באברי מין פנימיים בטיפול.\nסיכון מוגבר לאל-ווסת מוקדמת:\nנשאיות של פרה-מוטציה ל-X שביר;\nנוכחות סמנים המעלים סיכון לאל-וסת מוקדם;\nנשים הסובלות ממחלות אוטואימוניות, תסמונות כרומוזומליות או אחרות (המוכחות כמעלות סיכון למצב של אל-ווסת מוקדמת).\nנשים לפני ניתוח:\nכריתה מונעת של שחלות (נשאיות BRCA);\nניתוח שעלול להיות כרוך בכריתת שחלות.',
     'qas': []},
    {'context': 'החל משנת 2011 ניתן לבצע הקפאת ביציות גם ללא סיבות רפואיות.\nזכאיות לכך נשים המעונינות לשמר ביציות ואשר מלאו להן 30 שנים אך טרם מלאו להן 41.\nמספר הטיפולים המירבי בנשים במקרה כזה הוא עד 6 שאיבות.\nמספר הביציות המירבי שמותר לשאוב בין הגילאים 30-35 הוא 25 ביציות.\nמספר הביציות המירבי שמותר לשאוב בין הגילאים 36-40 הוא 35 ביציות.\nאם הושג מספר 

In [11]:
json_test_data

{'data': [{'title': 'מדריך_להתמודדות_עם_הטרדה_מינית_במסגרת_העבודה',
   'paragraphs': [{'context': 'אחד האתגרים בנושא הטרדה מינית הוא ההבחנה מתי חיזור שנתפס כלגיטימי הופך להטרדה מינית. המידע במדריך מתמודד עם השאלה ובעיקר בהדגשת זכות הסירוב של הקורבן להמשיך ולסבול מההתנהגות הפוגענית.\nמחקרים מלמדים שבמקומות עבודה שמתרחשות בהם הטרדות מיניות ובמקומות עם סביבת עבודה רווית תכנים מיניים, קיימת הסתברות גבוהה לאפליה בעבודה.\nהמדריך להתמודדות עם הטרדה מינית מיועד לסייע לנשים ולגברים שהוטרדו או מוטרדות/מוטרדים מינית במקום העבודה. המדריך מסייע גם למעסיקים לנהל סביבה מוגנת מהטרדות מיניות ולהתמודד עם אירועים של הטרדה מינית בהקשר של מקום העבודה.',
     'qas': []},
    {'context': 'הטרדה מינית יכולה להיות מילולית, לא מילולית (שפת גוף, רמיזות, הצגת תכנים פורנוגרפיים) ופיזית.\nהטרדה מינית נתפסת לרוב כמצב שבו נשים מוטרדות מינית על-ידי גברים, אבל עשויה לקרות גם כאשר נשים מטרידות גברים או כאשר נשים וגברים מוטרדים על-ידי בנות/בני מינן/ם.\nהחוק בישראל מחמיר בכל הנוגע להטרדה מינית במקומות עבודה ובמסגרת יחסי ע